# How to Connect to Coinbase Pro and Display Advanced Crypto Currency Trading Indicators?

by Nils Ackermann

## Introduction

In case you want to get started as a crypto investor and you want to take matters into your own hands, it is important that you are able to connect to a currency exchange such as [pro.coinbase.com](https://pro.coinbase.com) to pull market data. In this notebook you will learn the following:

- How to conveniently connect to coinbase pro via the cbpro Python library
- How to load data from coinbase pro into a pandas dataframe
- How to transform crypto currency data and add additional trading indicators (Simple Moving Average, Exponential Moving Average, MACD, MACD Signal, MACD Histogram) with the help of pandas
- How to visualize crypto currency market data via interactive charts using plotly

## Step 0 - Import Libraries and Set Constants

We will use very basic python libraries. The only "special" one that you might not be aware of is [cbpro](https://github.com/danpaquin/coinbasepro-python). This is a library which allows you to conveniently connect to the Coinbase Pro API. Before we connect, we will set a few parameters like the base currency (e.g. USD or EUR) that we want to use as well as a list of cryptos that we want to invest in. Feel free to modify the list with your cryptos. 

In [126]:
import cbpro
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from math import trunc
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [127]:
# Choose currency that should be used to display data
FIAT_CURRENCIES = ['EUR','USD']
MY_BASE_CURRENCY = FIAT_CURRENCIES[0]
# I am only interested in a few currencies that I want to trade, so let's add them here:
MY_CRYPTO_CURRENCIES = ["BTC","ETH","LTC","ALGO","SHIB","MANA"] 

RED_COLOR = '#D61170'
GREEN_COLOR = '#3FFA68'
MACD_COLOR = '#A9B1F5'
MACD_SIGNAL_COLOR = '#FFD1A3'
CANDLE_INCREASE_COLOR = 'cyan'
CANDLE_DECREASE_COLOR = 'gray'
#PLOTLY_TEMPLATE = 'plotly_dark'
PLOTLY_TEMPLATE = 'plotly'
GRANULARITIES = ['daily','60min','15min','1min']

## Step 1 - Establish Connection to Coinbase Pro

Connecting to the Coinbase API is super simple with the help of the cbpro library. For this notebook, we only need to connect to the public Coinbase Pro API, therefore no authentication is needed.

In [128]:
# Initiate public Coinbase Pro client
public_client = cbpro.PublicClient()
server_time = public_client.get_time()
# Server time does not comply to iso format, therefore slight modification of string needed
server_time_now = datetime.fromisoformat(server_time['iso'].replace('T', ' ', 1)[0:19])
print(server_time_now)
print(datetime.now())

2021-12-17 12:43:27
2021-12-17 13:43:27.103302


## Step 2 - Show Available Products (Crypto Currencies)

First, we will review some basic information about the crypto currencies that we chose to trade on Coinbase Pro.

In [129]:
df_currencies = pd.DataFrame(public_client.get_products())
print("\nNumber of columns in the dataframe: %i" % (df_currencies.shape[1]))
print("Number of rows in the dataframe: %i\n" % (df_currencies.shape[0]))
columns = list(df_currencies.columns)
df_currencies[df_currencies.id.isin(list(map(lambda orig_string: orig_string+'-'+MY_BASE_CURRENCY, MY_CRYPTO_CURRENCIES)))][['id', 'quote_currency', 'base_min_size', 'base_max_size']]


Number of columns in the dataframe: 20
Number of rows in the dataframe: 404



,id,quote_currency,base_min_size,base_max_size
23,LTC-EUR,EUR,0.0042,2900
89,SHIB-EUR,EUR,18000,3900000000
230,ALGO-EUR,EUR,1,190000
312,ETH-EUR,EUR,0.00021,610
354,BTC-EUR,EUR,0.000015,57
370,MANA-EUR,EUR,0.31,68000


In case you later want to place orders via the Coinbase Pro API (not covered in this notebook), it is important to provide the right level of precision for the order volume. So let's calculate the allowed digits after the decimal point for each of our currencies:

In [130]:
decimal_info_rows = []
for currency in MY_CRYPTO_CURRENCIES:
    product_id = currency+'-'+MY_BASE_CURRENCY
    min_size = float(df_currencies.query('id == @product_id')['base_min_size'])
    precision = min_size-trunc(min_size)
    if precision == 0:
        decimals = 0
    elif 1 > precision > 0.1:
        decimals = 1
    elif 0.1 > precision > 0.01:
        decimals = 2
    elif 0.01 > precision > 0.001:
        decimals = 3
    elif 0.001 > precision > 0.0001:
        decimals = 4
    else:
        decimals = 5
    row_data = [product_id,min_size,precision,decimals]
    decimal_info_rows.append(row_data)
df_decimal_info = pd.DataFrame(decimal_info_rows, columns = ['id','base_min_size','precision','allowed_decimals'])
df_decimal_info

,id,base_min_size,precision,allowed_decimals
0,BTC-EUR,0.000015,0.000015,5
1,ETH-EUR,0.000210,0.000210,4
2,LTC-EUR,0.004200,0.004200,3
3,ALGO-EUR,1.000000,0.000000,0
4,SHIB-EUR,18000.000000,0.000000,0
5,MANA-EUR,0.310000,0.310000,1


## Step 3 - Load 24h Stats

Next, we will review the basic stats from the last 24 hours for each of the crypto currencies that we have defined in our list above. We will also add a custom column "performance" which will show us the performance from the beginning to the end of the reporting period. We will also do some nice formatting of the numbers for printing purposes.

In [131]:
currency_rows = []
for currency in MY_CRYPTO_CURRENCIES:
    data = public_client.get_product_24hr_stats(currency+'-'+MY_BASE_CURRENCY)
    currency_rows.append(data)
df_24hstats = pd.DataFrame(currency_rows, index = MY_CRYPTO_CURRENCIES)
df_24hstats['currency'] = df_24hstats.index
df_24hstats['open'] = df_24hstats['open'].astype(float)
df_24hstats['high'] = df_24hstats['high'].astype(float)
df_24hstats['low'] = df_24hstats['low'].astype(float)
df_24hstats['volume'] = df_24hstats['volume'].astype(float)
df_24hstats['last'] = df_24hstats['last'].astype(float)
df_24hstats['volume_30day'] = df_24hstats['volume_30day'].astype(float)
df_24hstats['performance'] = ((df_24hstats['last']-df_24hstats['open']) / df_24hstats['open']) * 100
df_24hstats_formatted = df_24hstats.copy()
df_24hstats_formatted['performance'] = df_24hstats_formatted['performance'].apply(lambda x: "{:.2f}%".format((x)))
df_24hstats_formatted['open'] = df_24hstats_formatted['open'].apply(lambda x: "{:,.2f}€".format((x)))
df_24hstats_formatted['high'] = df_24hstats_formatted['high'].apply(lambda x: "{:,.2f}€".format((x)))
df_24hstats_formatted['low'] = df_24hstats_formatted['low'].apply(lambda x: "{:,.2f}€".format((x)))
df_24hstats_formatted['last'] = df_24hstats_formatted['last'].apply(lambda x: "{:,.2f}€".format((x)))
print('Performance within last 24 hours in ' + MY_BASE_CURRENCY + ':')
df_24hstats_formatted.style.hide_columns(['volume','volume_30day'])

Performance within last 24 hours in EUR:


,open,high,low,last,currency,performance
BTC,"43,022.78€","43,453.28€","41,300.00€","41,673.61€",BTC,-3.14%
ETH,"3,561.79€","3,642.97€","3,356.00€","3,384.20€",ETH,-4.99%
LTC,136.00€,137.63€,128.72€,130.10€,LTC,-4.34%
ALGO,1.27€,1.29€,1.20€,1.20€,ALGO,-5.22%
SHIB,0.00€,0.00€,0.00€,0.00€,SHIB,-5.37%
MANA,2.93€,2.95€,2.70€,2.72€,MANA,-6.97%


## Step 4 - Load Historic Data

Above, we have only looked at the last 24 hours. Now we will download data using four different time slices and periods:
- Daily data of the last 90 days
- Hourly data of the last 300 hours
- 15-minute data of the last 75 hours
- 1-minute data of the last 2 hours

All this data is stored in a pandas dataframe called df_history.

In [132]:
currency_history_rows = []
for currency in MY_CRYPTO_CURRENCIES:
    # Daily data:
    start_date = (server_time_now - timedelta(days=90)).isoformat()
    end_date = server_time_now.isoformat()
    data = public_client.get_product_historic_rates(currency+'-'+MY_BASE_CURRENCY, start=start_date, end=end_date, granularity=86400)
    [x.append(currency) for x in data]
    [x.append('daily') for x in data]
    currency_history_rows.extend(data)
    # 60 minutes data:
    start_date = (server_time_now - timedelta(hours=300)).isoformat()
    end_date = server_time_now.isoformat()
    data = public_client.get_product_historic_rates(currency+'-'+MY_BASE_CURRENCY, start=start_date, end=end_date, granularity=3600)
    [x.append(currency) for x in data]
    [x.append('60min') for x in data]
    currency_history_rows.extend(data)
    # 15 minutes data:
    start_date = (server_time_now - timedelta(hours=75)).isoformat()
    end_date = server_time_now.isoformat()
    data = public_client.get_product_historic_rates(currency+'-'+MY_BASE_CURRENCY, start=start_date, end=end_date, granularity=900)
    [x.append(currency) for x in data]
    [x.append('15min') for x in data]
    currency_history_rows.extend(data)
    # 1 minutes data:
    start_date = (server_time_now - timedelta(hours=2)).isoformat()
    end_date = server_time_now.isoformat()
    data = public_client.get_product_historic_rates(currency+'-'+MY_BASE_CURRENCY, start=start_date, end=end_date, granularity=60)
    [x.append(currency) for x in data]
    [x.append('1min') for x in data]
    currency_history_rows.extend(data)
df_history = pd.DataFrame(currency_history_rows)
# Add column names in line with the Coinbase Pro documentation
df_history.columns = ['time','low','high','open','close','volume','currency','granularity']
# We will add a few more columns just for better readability
df_history['date'] = pd.to_datetime(df_history['time'], unit='s')
df_history['year'] = pd.DatetimeIndex(df_history['date']).year
df_history['month'] = pd.DatetimeIndex(df_history['date']).month
df_history['day'] = pd.DatetimeIndex(df_history['date']).day
df_history['hour'] = pd.DatetimeIndex(df_history['date']).hour
df_history['minute'] = pd.DatetimeIndex(df_history['date']).minute

In [133]:
print(f'Daily data included in df_history (in {MY_BASE_CURRENCY}):')
df_history.query('granularity == \'daily\'').head(5).style.hide_index().hide_columns(['time','date','hour','minute'])

Daily data included in df_history (in EUR):


low,high,open,close,volume,currency,granularity,year,month,day
41300.000000,42399.770000,42033.840000,41656.810000,356.816466,BTC,daily,2021,12,17
41957.660000,43800.000000,43291.000000,42034.100000,781.309516,BTC,daily,2021,12,16
41372.120000,43866.610000,42950.000000,43288.220000,1312.536409,BTC,daily,2021,12,15
41020.000000,43250.000000,41408.060000,42955.020000,1215.250863,BTC,daily,2021,12,14
40515.700000,44403.760000,44302.590000,41389.360000,1594.526252,BTC,daily,2021,12,13


In [134]:
print(f'60 minute data included in df_history (in {MY_BASE_CURRENCY}):')
df_history.query('granularity == \'60min\'').tail(5).style.hide_index().hide_columns(['time','minute'])

60 minute data included in df_history (in EUR):


low,high,open,close,volume,currency,granularity,date,year,month,day,hour
3.481000,3.559000,3.494000,3.539000,123182.730000,MANA,60min,2021-12-05 05:00:00,2021,12,5,5
3.430000,3.514000,3.484000,3.493000,139552.320000,MANA,60min,2021-12-05 04:00:00,2021,12,5,4
3.409000,3.509000,3.430000,3.481000,111933.340000,MANA,60min,2021-12-05 03:00:00,2021,12,5,3
3.408000,3.503000,3.484000,3.424000,136390.280000,MANA,60min,2021-12-05 02:00:00,2021,12,5,2
3.429000,3.516000,3.483000,3.485000,217089.600000,MANA,60min,2021-12-05 01:00:00,2021,12,5,1


In [135]:
print(f'15 minute data included in df_history (in {MY_BASE_CURRENCY}):')
df_history.query('granularity == \'15min\'').tail(5).style.hide_index().hide_columns(['time'])

15 minute data included in df_history (in EUR):


low,high,open,close,volume,currency,granularity,date,year,month,day,hour,minute
2.802000,2.824000,2.808000,2.824000,6402.260000,MANA,15min,2021-12-14 10:45:00,2021,12,14,10,45
2.802000,2.840000,2.818000,2.806000,12441.810000,MANA,15min,2021-12-14 10:30:00,2021,12,14,10,30
2.810000,2.827000,2.813000,2.816000,8125.710000,MANA,15min,2021-12-14 10:15:00,2021,12,14,10,15
2.803000,2.817000,2.806000,2.816000,1758.620000,MANA,15min,2021-12-14 10:00:00,2021,12,14,10,0
2.799000,2.821000,2.809000,2.801000,2267.900000,MANA,15min,2021-12-14 09:45:00,2021,12,14,9,45


In [136]:
print(f'1 minute data included in df_history (in {MY_BASE_CURRENCY}):')
df_history.query('granularity == \'1min\'').tail(5).style.hide_index().hide_columns(['time'])

1 minute data included in df_history (in EUR):


low,high,open,close,volume,currency,granularity,date,year,month,day,hour,minute
2.728000,2.735000,2.735000,2.729000,2519.380000,MANA,1min,2021-12-17 10:49:00,2021,12,17,10,49
2.735000,2.737000,2.736000,2.735000,629.650000,MANA,1min,2021-12-17 10:48:00,2021,12,17,10,48
2.728000,2.735000,2.728000,2.735000,1289.030000,MANA,1min,2021-12-17 10:47:00,2021,12,17,10,47
2.723000,2.726000,2.726000,2.724000,324.840000,MANA,1min,2021-12-17 10:45:00,2021,12,17,10,45
2.720000,2.726000,2.720000,2.725000,864.690000,MANA,1min,2021-12-17 10:44:00,2021,12,17,10,44


## Step 5 - Get More Insights

Let's add a few more parameters to the basic data (residing in df_history):

- **SMA3 and SMA7 and SMA12** (Simple Moving Average over last 3 and 7 time slices): This is a price based, lagging (or reactive) indicator that displays the average price of a security over a certain period of time. Moving Averages smooth out the “noise” when interpreting charts. Noise is made up of fluctuations of both price and volume. Because a Moving Average is a lagging indicator and reacts to events that have already happened, it is not used as a predictive indicator but rather an interpretive one. Simple Moving Average is an unweighted Moving Average. This means that each period in the data set has equal importance and is weighted equally. 
- **EMA12 and EMA26** (Exponential Moving Average over last 12 and 26 time slices): The EMA has much less lag than the SMA because it puts a greater importance on more recent prices. It therefore turns quicker than the SMA. You can choose different time slices, but 12 and 26 are commonly used durations.
- **MACD** (Moving Average Convergence/Divergence): A great indicator to identify the overall trend of any security (e.g. crypto currencies). It takes the difference between the short term EMA and the long term EMA. A positive MACD value is an indicator for a positive market trend. A negative MACD value is an indicator for a negative market trend. 
- **MACD Signal**: The MACD Signal line is the EMA over a certain set of time slices of the MACD line. Typically, the value is set to 9 time slices. 
- **MACD Histogram**: It is the difference between the MACD line and the MACD Signal line. A bullish crossover occurs when the MACD line crosses above the MACD Signal line. A bearish crossover occurs if the MACD line crosses below the MACD Signal line. 
- **Open to close performance**: For each time slice, the difference between the closing price and the opening price of that particular period is given as a percentage.
- **High low span**: The deviation within one period between the highest and lowest price as a percentage. This shows the volatility within one time slice.
- **Absolute performance of last 3 periods**: This indicator provides the performance over the last three time slices as an absolute value.
- **Performance of last 3 periods in percentage**: This indicator provides the performance over the last three time slices as a relative value.
- **Bull or Bear market**: It simply writes "Bull" in case the MACD Histogram is positive and it writes "Bear" in case the MACD Histogram value is negative. 
- **Market trend continued**: To identify a shift from Bull to Bear market and vice versa, this column holds "True" in case the trend from the previous time slice continues and "False" in case a trend change has occurred. 

The enhanced data is stored in a pandas dataframe called df_history_enhanced. We will again use the four different granularities defined above.

In case you want to learn more about the different trading indicators, take a look [here](https://www.tradingview.com).

In [137]:
currency_history_rows_enhanced = []
for currency in MY_CRYPTO_CURRENCIES:
    for granularity in GRANULARITIES:
        df_history_currency = df_history.query('granularity == @granularity & currency == @currency').copy()
        # Oldest to newest date sorting needed for SMA calculation
        df_history_currency = df_history_currency.sort_values(['date'], ascending=True)
        df_history_currency['SMA3'] = df_history_currency['close'].rolling(window=3).mean()
        df_history_currency['SMA7'] = df_history_currency['close'].rolling(window=7).mean()
        df_history_currency['EMA12'] = df_history_currency['close'].ewm(span=12, adjust=False).mean()
        df_history_currency['EMA26'] = df_history_currency['close'].ewm(span=26, adjust=False).mean()
        df_history_currency['MACD'] = df_history_currency['EMA12'] - df_history_currency['EMA26']
        df_history_currency['MACD_signal'] = df_history_currency['MACD'].ewm(span=9, adjust=False).mean()
        df_history_currency['macd_histogram'] = ((df_history_currency['MACD']-df_history_currency['MACD_signal']))
        df_history_currency['open_to_close_perf'] = ((df_history_currency['close']-df_history_currency['open']) / df_history_currency['open'])
        df_history_currency['high_low_span'] = ((df_history_currency['high']-df_history_currency['low']) / df_history_currency['high'])
        df_history_currency['open_perf_last_3_period_abs'] = df_history_currency['open'].rolling(window=4).apply(lambda x: x.iloc[1] - x.iloc[0])
        df_history_currency['open_perf_last_3_period_per'] = df_history_currency['open'].rolling(window=4).apply(lambda x: (x.iloc[1] - x.iloc[0])/x.iloc[0])
        df_history_currency['bull_bear'] = np.where(df_history_currency['macd_histogram']< 0, 'Bear', 'Bull')
        currency_history_rows_enhanced.append(df_history_currency)
df_history_enhanced = pd.concat(currency_history_rows_enhanced, ignore_index=True)
# Last step to tag changes in market trends from one period to the other (sorting important)
df_history_enhanced = df_history_enhanced.sort_values(['currency','granularity','date'], ascending=True)
df_history_enhanced['market_trend_continued'] = df_history_enhanced.bull_bear.eq(df_history_enhanced.bull_bear.shift()) & df_history_enhanced.currency.eq(df_history_enhanced.currency.shift()) & df_history_enhanced.granularity.eq(df_history_enhanced.granularity.shift())
df_history_enhanced.head(10)

,time,low,high,open,close,volume,currency,granularity,date,year,...,EMA26,MACD,MACD_signal,macd_histogram,open_to_close_perf,high_low_span,open_perf_last_3_period_abs,open_perf_last_3_period_per,bull_bear,market_trend_continued
2813,1639475100,1.1798,1.1877,1.1841,1.1824,18930.0,ALGO,15min,2021-12-14 09:45:00,2021,...,1.182400,0.000000,0.000000,0.000000,-0.001436,0.006652,NaN,NaN,Bull,False
2814,1639476000,1.1822,1.1961,1.1822,1.1920,13028.0,ALGO,15min,2021-12-14 10:00:00,2021,...,1.183111,0.000766,0.000153,0.000613,0.008290,0.011621,NaN,NaN,Bull,True
2815,1639476900,1.1882,1.1953,1.1928,1.1950,19494.0,ALGO,15min,2021-12-14 10:15:00,2021,...,1.183992,0.001596,0.000442,0.001155,0.001844,0.005940,NaN,NaN,Bull,True
2816,1639477800,1.1923,1.2015,1.1944,1.1937,30084.0,ALGO,15min,2021-12-14 10:30:00,2021,...,1.184711,0.002125,0.000778,0.001347,-0.000586,0.007657,-0.0019,-0.001605,Bull,True
2817,1639478700,1.1946,1.2032,1.1946,1.1985,18349.0,ALGO,15min,2021-12-14 10:45:00,2021,...,1.185732,0.002898,0.001202,0.001696,0.003265,0.007148,0.0106,0.008966,Bull,True
2818,1639479600,1.1950,1.2007,1.1995,1.1972,20842.0,ALGO,15min,2021-12-14 11:00:00,2021,...,1.186582,0.003367,0.001635,0.001732,-0.001917,0.004747,0.0016,0.001341,Bull,True
2819,1639480500,1.1936,1.2020,1.1967,1.1936,14931.0,ALGO,15min,2021-12-14 11:15:00,2021,...,1.187102,0.003409,0.001990,0.001419,-0.002590,0.006988,0.0002,0.000167,Bull,True
2820,1639481400,1.1923,1.1998,1.1937,1.1977,21500.0,ALGO,15min,2021-12-14 11:30:00,2021,...,1.187887,0.003730,0.002338,0.001392,0.003351,0.006251,0.0049,0.004102,Bull,True
2821,1639482300,1.1937,1.2110,1.1985,1.2110,39397.0,ALGO,15min,2021-12-14 11:45:00,2021,...,1.189599,0.005000,0.002870,0.002129,0.010430,0.014286,-0.0028,-0.002334,Bull,True
2822,1639483200,1.2104,1.2217,1.2109,1.2165,17962.0,ALGO,15min,2021-12-14 12:00:00,2021,...,1.191591,0.006377,0.003572,0.002805,0.004625,0.009249,-0.0030,-0.002507,Bull,True


## Step 6 - Print Out Selected Information

Now, we can leverage the data which is available in df_history_enhanced to compile some basic parameters that we might want to use in a personal dashboard down the line.

In [138]:
additional_info_rows = []
for currency in MY_CRYPTO_CURRENCIES:
    df_history_enhanced = df_history_enhanced.sort_values(['date'], ascending=True) #oldest to newest
    row_data = [currency,
        float(df_24hstats.query('currency == @currency')['last'].to_string(index=False)), # current value
        df_history_enhanced.query('granularity == \'1min\' and currency == @currency').tail(1)['bull_bear'].to_string(index=False), # 1min market trend
        df_history_enhanced.query('granularity == \'1min\' and currency == @currency').iloc[-2]['bull_bear'], # prev 1min market trend
        df_history_enhanced.query('granularity == \'15min\' and currency == @currency').tail(1)['bull_bear'].to_string(index=False), #15min market trend
        df_history_enhanced.query('granularity == \'15min\' and currency == @currency').iloc[-2]['bull_bear'], # prev 15min market trend
        df_history_enhanced.query('granularity == \'60min\' and currency == @currency').tail(1)['bull_bear'].to_string(index=False), # 60min market trend
        df_history_enhanced.query('granularity == \'60min\' and currency == @currency').iloc[-2]['bull_bear'], # prev 60min market trend
        df_history_enhanced.query('granularity == \'60min\' and currency == @currency').tail(24)['low'].min(), # lowest last 24h
        df_history_enhanced.query('granularity == \'daily\' and currency == @currency').tail(30)['low'].min(), # lowest last 30d
        df_history_enhanced.query('granularity == \'daily\' and currency == @currency').tail(90)['low'].min(), # lowest last 90d
        df_history_enhanced.query('granularity == \'60min\' and currency == @currency').tail(24)['high'].max(), # highest last 24h
        df_history_enhanced.query('granularity == \'daily\' and currency == @currency').tail(30)['high'].max(), # highest last 30d
        df_history_enhanced.query('granularity == \'daily\' and currency == @currency').tail(90)['high'].max(), # highest last 90d
        round(df_history_enhanced.query('granularity == \'daily\' and currency == @currency').tail(90)['high_low_span'].max(),3), # max daily high/low difference last 90d
        round(df_history_enhanced.query('granularity == \'60min\' and currency == @currency')['high_low_span'].max(),3) # max daily high/low difference last 12.5d
        ]
    additional_info_rows.append(row_data)
df_additional_info = pd.DataFrame(additional_info_rows, columns = ['currency',
                                                                    'current_val',
                                                                    '1min_trend',
                                                                    '1min_prev_trend',
                                                                    '15min_trend',
                                                                    '15min_prev_trend',
                                                                    '60min_trend',
                                                                    '60min_prev_trend',
                                                                    '24h_low',
                                                                    '30day_low',
                                                                    '90day_low',
                                                                    '24h_high',
                                                                    '30day_high',
                                                                    '90day_high',
                                                                    '90day_max_high_low_span',
                                                                    '12day_max_high_low_span'])
df_additional_info['current_val'] = df_additional_info['current_val'].apply(lambda x: "{:,.2f}€".format((x)))
df_additional_info['24h_low'] = df_additional_info['24h_low'].apply(lambda x: "{:,.2f}€".format((x)))
df_additional_info['30day_low'] = df_additional_info['30day_low'].apply(lambda x: "{:,.2f}€".format((x)))
df_additional_info['90day_low'] = df_additional_info['90day_low'].apply(lambda x: "{:,.2f}€".format((x)))
df_additional_info['24h_high'] = df_additional_info['24h_high'].apply(lambda x: "{:,.2f}€".format((x)))
df_additional_info['30day_high'] = df_additional_info['30day_high'].apply(lambda x: "{:,.2f}€".format((x)))
df_additional_info['90day_high'] = df_additional_info['90day_high'].apply(lambda x: "{:,.2f}€".format((x)))
df_additional_info['90day_max_high_low_span'] = (df_additional_info['90day_max_high_low_span']*100).apply(lambda x: "{:.2f}%".format((x)))
df_additional_info['12day_max_high_low_span'] = (df_additional_info['12day_max_high_low_span']*100).apply(lambda x: "{:.2f}%".format((x)))
df_additional_info.set_index('currency').transpose()

currency,BTC,ETH,LTC,ALGO,SHIB,MANA
current_val,"41,673.61€","3,384.20€",130.10€,1.20€,0.00€,2.72€
1min_trend,Bear,Bear,Bear,Bear,Bull,Bear
1min_prev_trend,Bear,Bear,Bear,Bull,Bull,Bear
15min_trend,Bull,Bull,Bull,Bull,Bear,Bear
15min_prev_trend,Bull,Bull,Bull,Bull,Bear,Bear
60min_trend,Bear,Bear,Bear,Bear,Bear,Bear
60min_prev_trend,Bear,Bear,Bear,Bear,Bear,Bear
24h_low,"41,300.00€","3,356.00€",128.72€,1.20€,0.00€,2.70€
30day_low,"39,505.01€","3,203.90€",120.00€,1.16€,0.00€,2.50€
90day_low,"33,768.45€","2,260.06€",119.71€,1.16€,0.00€,0.51€


## Step 7 - Visualize Data

For visualization, we will use the powerful [plotly](https://plotly.com) library. It will give us the possibility to slice and dice data in an interactive way. Exactly what we need for an in-depth analysis of our daily market data.

### Simple Visualization

First, let's create a simple OHLC (Open, High, Low, Close) candle stick chart for each of our currencies.

In [139]:
df_history_enhanced = df_history_enhanced.sort_values(['date'], ascending=True) #oldest to newest for visualization
granularity = '1min'
for currency in MY_CRYPTO_CURRENCIES:
    df_history_for_chart = df_history_enhanced.query(f'granularity == \'{granularity}\' and currency == \'{currency}\'')
    fig = go.Figure()
    fig.add_trace(go.Candlestick(
        x=df_history_for_chart['date'],
        open=df_history_for_chart['open'],
        high=df_history_for_chart['high'],
        low=df_history_for_chart['low'],
        close=df_history_for_chart['close'],
        name='OHLC'))
    cs = fig.data[0]
    cs.increasing.fillcolor = CANDLE_INCREASE_COLOR
    cs.increasing.line.color = CANDLE_INCREASE_COLOR
    cs.decreasing.fillcolor = CANDLE_DECREASE_COLOR
    cs.decreasing.line.color = CANDLE_DECREASE_COLOR
    fig.update_layout(
        xaxis_rangeslider_visible=False,
        title=f'OHLC Chart for {currency} with time interval {granularity}',
        template = PLOTLY_TEMPLATE,)
    fig.show()

### Advanced Visualization

Now that we have drawn a basic OHLC chart, let's get a bit more fancy. We will now draw the chart again. But this time we will add the following information:
- MACD line
- MACD signal line
- Colored area to show if we are currently in a bull market (green) or bear market (red) in each given period

Before we do that, there is one more important step to take: we must construct the data needed to color different periods for the market trends (bull versus bear). Therefore, let's create another pandas dataframe df_history_market_trend_intervals.

In [140]:
market_trend_interval_rows = []
for currency in MY_CRYPTO_CURRENCIES:
    for granularity in GRANULARITIES:
        df_history_market_trend_intervals = df_history_enhanced.query('currency == @currency and market_trend_continued == False and granularity == @granularity').copy()
        #df_history_market_trend_intervals['previous_period_date'] = df_history_market_trend_intervals.date.shift()
        df_history_market_trend_intervals['next_period_date'] = df_history_market_trend_intervals.date.shift(-1)
        df_history_market_trend_intervals['next_period_date'] = df_history_market_trend_intervals['next_period_date']
        df_history_market_trend_intervals.next_period_date = df_history_market_trend_intervals.next_period_date.fillna(datetime.now())
        df_history_market_trend_intervals['color'] = df_history_market_trend_intervals['bull_bear'].apply(lambda x: GREEN_COLOR if x == 'Bull' else RED_COLOR)
        df_history_market_trend_intervals = df_history_market_trend_intervals[['currency','granularity','bull_bear','color','date','next_period_date']].rename(columns={"date": "start_date", "next_period_date": "finish_date"})
        market_trend_interval_rows.append(df_history_market_trend_intervals)
df_history_market_trend_intervals = pd.concat(market_trend_interval_rows, ignore_index=True)

In [141]:
granularity = '1min'
for currency in MY_CRYPTO_CURRENCIES:
    df_history_for_chart = df_history_enhanced.query(f'granularity == \'{granularity}\' and currency == \'{currency}\'').copy()
    df_history_market_trend_for_chart = df_history_market_trend_intervals.query(f'granularity == \'{granularity}\' and currency == \'{currency}\'').copy()
    macd_min = df_history_for_chart['MACD'].min()
    macd_max = df_history_for_chart['MACD'].max()
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Ohlc(
        x=df_history_for_chart['date'],
        open=df_history_for_chart['open'],
        high=df_history_for_chart['high'],
        low=df_history_for_chart['low'],
        close=df_history_for_chart['close'],
        increasing_line_color= CANDLE_INCREASE_COLOR,
        decreasing_line_color= CANDLE_DECREASE_COLOR,
        name='OHLC'),
        secondary_y=True),
    fig.add_trace(go.Scatter(
        x=df_history_for_chart['date'],
        y=df_history_for_chart['MACD'],
        name='MACD',
        line=dict(color=MACD_COLOR),
        mode='lines'),
        secondary_y=False)
    fig.add_trace(go.Scatter(
        x=df_history_for_chart['date'],
        y=df_history_for_chart['MACD_signal'],
        name='MACD Signal',
        line=dict(color=MACD_SIGNAL_COLOR),
        mode='lines'),
        secondary_y=False)
    for i in range(df_history_market_trend_for_chart['currency'].count()):
        fig.add_vrect(x0=df_history_market_trend_for_chart.iloc[i]['start_date'],
            x1=df_history_market_trend_for_chart.iloc[i]['finish_date'],
            col=1,
            #fillcolor=df_history_market_trend_for_chart.iloc[i]['color'],
            line=dict(color=df_history_market_trend_for_chart.iloc[i]['color']),
            opacity=1.0,
            line_width=1)
    fig.update_layout(
        title=f'OHLC Chart for {currency} with time interval {granularity}',
        hovermode='x unified',
        legend=dict(yanchor="top",y=0.99,xanchor="right",x=0.94),
        #autosize=False,
        #width=900,
        #height=600,
        template = PLOTLY_TEMPLATE,
        yaxis_range=[macd_min,macd_max],
        yaxis1_showgrid = False,
        yaxis1_showticklabels = False,
        yaxis1_zeroline = False,
        margin=dict(
            l=50,
            r=50,
            b=100,
            t=100,
            pad=4
        ),
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=1,
                        label="1m",
                        step="month",
                        stepmode="backward"),
                    dict(count=6,
                        label="6m",
                        step="month",
                        stepmode="backward"),
                    dict(count=1,
                        label="YTD",
                        step="year",
                        stepmode="todate"),
                    dict(count=1,
                        label="1y",
                        step="year",
                        stepmode="backward"),
                    dict(step="all")
                ])
            ),
            rangeslider=dict(visible=True),
            type="date"
        )
    )
    if granularity == 'daily':
        fig.update_layout(xaxis_tickformat = '%d %b (%a)')
    else:
        fig.update_layout(xaxis_tickformat = '%d %b %H:%M')
    fig.show()